In [ ]:
# default_exp sensors

# Sensors
> reads sensor data over UART


In [ ]:
#export
import time
import serial
import numpy as np
import pandas as pd

from pathlib import Path
#import Jetson.GPIO as GPIO
import datetime
import pickle


Sensor data packet read and decode pyserial code below. Modified from a past project of mine. 

Todo: Read the RTC timestamp at beginning of collection. The data packets only contain timestamp offsets.


In [ ]:
#hardware

def read_packet(header:chr = "*", num_bytes:int = 80, timeout:float = 6.) -> "byte string":
    """Reads `num_bytes` of a data packet starting with `header` and timing out after `timeout` seconds if packet is invalid."""
    buff = b""
    
    # Check for packet start of frame
    start_time = time.time()
    while True:
        if ser.in_waiting> 0:
            b = ser.read()
            if b ==  b'*':
                #print("Received Packet")
                break
            
        if time.time() > start_time + timeout: 
            print("No data packets.")
            return None
    
    # read the rest of data packet
    start_time = time.time()
    for i in range(num_bytes):
        if time.time() > start_time + timeout:
            print("Received Incomplete Packet")
            break
        
        if ser.in_waiting > 0:
            buff += ser.read()
            
    return buff

def decode_packet(buff:"byte string") -> list:
    """Decode `buff` into a list of decoded variables"""
    if buff is None:
        return []
    
    contents = []
    np_buff = np.frombuffer(buff,dtype='uint8').astype(np.uint8)
    
    # Real Time Clock
    contents.append( np_buff[0:4].view(np.uint32)    ) # rtc timestamp offset [ms]
    
    # Humidity, Pressure sensor
    contents.append( np_buff[4:8].view(np.uint32)    ) # humidity timestamp offset [ms]
    contents.append( np_buff[8:12].view(np.float32)  ) # temperature [deg C]
    contents.append( np_buff[12:16].view(np.float32) ) # pressure [hPa]
    contents.append( np_buff[16:20].view(np.float32) ) # humidity [relative humidity %]
    
    # Inertial Measurement Unit
    contents.append( np_buff[20:24].view(np.uint32)  ) # imu timestamp offset [ms]
    contents.append( np_buff[24:28].view(np.float32) ) # quaternion w
    contents.append( np_buff[28:32].view(np.float32) ) # quaternion x
    contents.append( np_buff[32:36].view(np.float32) ) # quaternion y
    contents.append( np_buff[36:40].view(np.float32) ) # quaternion z
    contents.append( np_buff[40:41].view(np.uint8)   ) # calibration status
    
    # Global Positioning System
    contents.append( np_buff[41:45].view(np.uint32)  ) # gps timestamp offset [ms]
    contents.append( np_buff[45:49].view(np.int32)   ) # latitude [deg *10^-7]
    contents.append( np_buff[49:53].view(np.int32)   ) # longitude [deg *10^-7]
    contents.append( np_buff[53:57].view(np.int32)   ) # altitude [mm above ellipsoid]
    contents.append( np_buff[57:58].view(np.uint8)   ) # number of satellites in view and used in compute
    contents.append( np_buff[58:62].view(np.int32)   ) # ground speed [mm/s]
    contents.append( np_buff[62:66].view(np.int32)   ) # heading of motion [deg *10^-5]
    contents.append( np_buff[66:68].view(np.uint16)  ) # position DOP [*0.01]
    contents.append( np_buff[68:72].view(np.uint32)  ) # horizontal accuracy estimate [mm]
    contents.append( np_buff[72:76].view(np.uint32)  ) # vertical accuracy estimate [mm]
    contents.append( np_buff[76:80].view(np.uint32)  ) # heading accuracy estimate (both motion and vehicle) [deg *10^-5]
    
    return contents



In [ ]:
#export

# Define dictionary column name and data type
fields_dict = {'rtc_now': 'datetime', 
             'rtc_temp': 'float', 
             'air_temp': 'float', 
             'air_pressure': 'float', 
             'air_humidity': 'float', 
             'imu_cal': 'int',
             'imu_temp': 'float', 
             'euler_x': 'float', 
             'euler_y': 'float', 
             'euler_z': 'float', 
             
             'quat0': 'float',
             'quat1': 'float',
             'quat2': 'float',
             'quat3': 'float',
             
             'mag_x': 'float',
             'mag_y': 'float', 
             'mag_z': 'float', 
             'gps_now': 'datetime', 
             'latitude': 'float', 
             'longitude': 'float', 
             'altitude': 'float', 
             'numSV': 'int', 
             'velN': 'float', 
             'velE': 'float', 
             'gSpeed': 'float', 
             'heading': 'float', 
             'velAcc': 'float', 
             'pDOP': 'float', 
             'hAcc': 'float', 
             'vAcc': 'float', 
             'headAcc': 'float', 
             'magDec': 'float', 
             'magAcc': 'float'}

In [ ]:
#export

class SensorStream():

    def __init__(self, baudrate=921_600, port="/dev/ttyTHS0", start_pin=27, save_dir="/xavier_ssd/data/"):

        self.ser = serial.Serial(port=port,
                                baudrate=baudrate,
                                bytesize=serial.EIGHTBITS,
                                parity=serial.PARITY_NONE,
                                stopbits=serial.STOPBITS_ONE,
                                )

        # Initialise serial port and wait
        self.ser.flushInput()

        # Instantiate for storing data
        self.data = []
        self.data_df = None

        self.start_pin = start_pin

        #GPIO.setwarnings(False)
        GPIO.setmode(GPIO.BCM) # BCM pin-numbering scheme from Raspberry Pi
        GPIO.setup(start_pin, GPIO.IN)
        self.dir = Path(f"{save_dir}{datetime.date.today()}")
        self.dir.mkdir(parents=False, exist_ok=True)
        self.fname = (f"{self.dir}/{datetime.datetime.now()}.pkl")
        
    def save(self):
        self.fname = (f"{self.dir}/{datetime.datetime.now()}.pkl")
        self.to_df(fields_dict,dropna_subset = ['rtc_now'],save_file=self.fname)
        print(f"Saved {len(self.data)} lines to {self.fname}")
        self.data = []
        print(self.data_df) #print(self.data_df.head(1)); print(self.data_df.tail(1))
        #self.ser.flushInput()
        

    def run(self):
        print("starting sensor datapacket reads")
        while True:
            try:
                if GPIO.input(self.start_pin) == True:
                    if len(self.data) == 0:
                        print("packets are coming")
                    self.record()
                    
                    if len(self.data) > 2**14: # about 14k
                        self.save()
                    
                else:
                    if len(self.data) > 0:
                        self.save()
                
                time.sleep(1)

            except KeyboardInterrupt:
                GPIO.cleanup()
                print("Exiting sensor read.")
                break
            except Exception as e:
                print(e)
                print("Attempting to start again!")
                self.ser.flushInput()
                self.data = []



    def record(self,max_timeout=2):

        start_time = time.time()

        # Check if line is ready
        while self.ser.inWaiting() > 0:

            # Read line from serial
            line_data = self.ser.readline()

            # Format data
            line_data = str(line_data).replace("b", "").replace("'", "").split(",")[:-1]

            # Append line to list
            self.data.append(line_data)

            if time.time()-start_time > max_timeout:
                print("timeout")
                self.ser.flushInput()
                break


    def to_df(self, fields_dict, dropna_subset = None, save_file = None):

        # Convert to dataframe and drop the first row of unclean data
        self.data_df = pd.DataFrame(self.data[1:], columns = fields_dict.keys())
        if dropna_subset:
            self.data_df.dropna(subset = dropna_subset, inplace=True)

        # Iterate through each column and update data type
        for field_name in fields_dict.keys():

            # Convert to float
            if fields_dict[field_name] == 'float' or fields_dict[field_name] == 'int':
                self.data_df[field_name] =  pd.to_numeric(self.data_df[field_name], errors='coerce')

            # Convert to datetime
            elif fields_dict[field_name] == 'datetime':
                self.data_df[field_name] =  pd.to_datetime(self.data_df[field_name], errors='coerce')

        if save_file is not None:
            with open(save_file,"wb") as handle:
                pickle.dump(self.data_df,handle,protocol=pickle.HIGHEST_PROTOCOL)

        # Return formatted data as dataframe
        return self.data_df

In [ ]:
#hardware

peripherals = SensorStream(baudrate=921_600,port="/dev/ttyTHS0")
peripherals.run() # start on switch on, pause on switch off, KeyboardInterrupt to stop



In [ ]:
#hardware

peripherals = SensorStream(baudrate=921_600,port="/dev/ttyTHS0")
peripherals.record(max_timeout=2)
print(peripherals.data)

In [ ]:
#test
print("test")


test


ValueError: Test flag entered

In [ ]:
#hardware
print("hardware")
raise ValueError("Hardware flag entered")

hardware


In [ ]:
#skip
print("skip")

skip
